In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_dataset
# import evaluate  # pip install evaluate
# import seqeval   # pip install seqeval
from datasets import Dataset
import numpy as np
from transformers import pipeline

2025-06-20 02:28:52.784065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750386532.940170      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750386532.984150      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
ds = load_dataset('doushabao4766/msra_ner_k_V3')
ds

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

In [3]:
for data in ds['train']:
    print(data)
    print(data['tokens'])
    print(data['ner_tags'])
    print(len(data['tokens']))
    print(len(data['ner_tags']))
    break

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': ''}
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
50
50


In [4]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [5]:
tags_id= set()
for data in ds['train']:
    ner_tags = data['ner_tags']
    tags_id.update(ner_tags)
print(tags_id)

{0, 1, 2, 3, 4, 5, 6}


In [6]:
entity_types = ['0'] + list({'PER','ORG','LOC'})
tags = ['0']
for entity in entity_types[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i,entity in enumerate(entity_types)}
entity_index

{'0': 0, 'PER': 1, 'LOC': 2, 'ORG': 3}

In [7]:
input_data = tokenizer(ds['train'][1:3]['tokens'],
                           truncation=True,
                           add_special_tokens=False,
                           max_length=512, 
                           is_split_into_words=True)
input_data

{'input_ids': [[5966, 741, 3315, 3341, 2218, 3221, 2792, 3300, 837, 5320, 3119, 5966, 7305, 5102, 704, 4638, 5018, 671, 1920, 2787, 8024, 1372, 3221, 2769, 812, 5310, 3338, 3946, 7653, 4638, 3198, 7313, 1922, 4764, 5445, 2347, 511], [1728, 3300, 1068, 3189, 2167, 1762, 776, 2966, 1932, 3152, 4289, 6422, 2658, 8024, 5966, 4518, 6772, 711, 7028, 6228, 8024, 738, 3221, 2769, 812, 3119, 5966, 1266, 776, 1380, 3160, 704, 4638, 6206, 816, 722, 671, 511]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [8]:
def data_input_proc(item):
    input_data = tokenizer(item['tokens'],
                           truncation=True,
                           add_special_tokens=False,
                           max_length=512, 
                           is_split_into_words=True)

    input_data['labels'] = [lbl[:512] for lbl in item['ner_tags']]
    return input_data

ds1 = ds.map(data_input_proc,batched=True)
ds1.set_format('torch',columns = ['input_ids','token_type_ids','attention_mask','labels'])

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [9]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})

In [10]:
args = TrainingArguments(
    output_dir="msra_ner_train",
    num_train_epochs = 3,
    # save_safetensors = False,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=300,
    fp16=True, #使用混合精度
    lr_scheduler_type='linear', #动态学习率
    warmup_steps=100,  #预热步数
    ddp_find_unused_parameters=False #优化DDP性能
)

In [11]:
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

ner_model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                           num_labels=len(tags),
                                                           id2label=id2lbl,
                                                           label2id=lbl2id)
ner_model

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [12]:
def compute_metric(result):
    # 这是 Trainer 的约定，result 一定是一个 (predictions, labels) 的元组，所以可以直接解包。
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(prdicts,axis=-1)

    predicts = [[tags[p] for p,l in zip(ps,ls) if l != 100]
        for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != 100]
        for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)
    return results

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,padding=True)

trainer = Trainer(
    model=ner_model,
    args=args,
    train_dataset=ds1['train'],
    eval_dataset=ds1['test'],
    data_collator=data_collator
)

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
